In [3]:
#imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#gpu config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [5]:
#hyperparams
learning_rate = 0.01
batch_size = 128
num_epochs = 30

#load data
transform = transforms.Compose([transforms.ToTensor()])
train_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

100%|██████████| 170498071/170498071 [00:04<00:00, 34623482.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
#LeNet5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
#initialize
model = LeNet5().to(device)
writer = SummaryWriter()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
#Training

#train net
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        
        # Send data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass and compute loss
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute running train loss
        train_loss += loss.item()
        
    #validation
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    valid_loss = valid_loss/len(test_loader.dataset)
    accuracy = 100 * correct / total

    writer.add_scalar('Training Loss', train_loss / len(train_loader), epoch)
    writer.add_scalar('Validation Loss', valid_loss / len(test_loader), epoch)
    
    print(f"Epoch {epoch+1}/{30}, Training Loss: {train_loss:.6f}, Validation Loss: {valid_loss:.6f}, Accuracy: {accuracy:.2f}")
writer.close()

Epoch 1/30, Training Loss: 900.263603, Validation Loss: 2.299963, Accuracy: 12.95
Epoch 2/30, Training Loss: 898.066054, Validation Loss: 2.292199, Accuracy: 12.86
Epoch 3/30, Training Loss: 891.406126, Validation Loss: 2.256739, Accuracy: 16.08
Epoch 4/30, Training Loss: 863.389615, Validation Loss: 2.166972, Accuracy: 19.50
Epoch 5/30, Training Loss: 821.552043, Validation Loss: 2.046817, Accuracy: 25.38
Epoch 6/30, Training Loss: 780.681578, Validation Loss: 1.963592, Accuracy: 28.99
Epoch 7/30, Training Loss: 756.562629, Validation Loss: 1.891708, Accuracy: 31.53
Epoch 8/30, Training Loss: 734.047255, Validation Loss: 1.873262, Accuracy: 31.87
Epoch 9/30, Training Loss: 715.489771, Validation Loss: 1.814458, Accuracy: 34.10
Epoch 10/30, Training Loss: 694.380733, Validation Loss: 1.742715, Accuracy: 37.27
Epoch 11/30, Training Loss: 674.283748, Validation Loss: 1.666976, Accuracy: 39.86
Epoch 12/30, Training Loss: 654.398649, Validation Loss: 1.631593, Accuracy: 41.47
Epoch 13/30, 